In [14]:
def parse_db_name(source):
    start = source.find("from")
    source = source[start + 4:]
    end = source.find("</a")
    source = source[:end]
    return source.lstrip().rstrip()


def parse_entry_name(source):
    href = ''

    if source.find("href") != -1:
        """
            For the entry which has the web id
            """
        start = source.find("href")
        _temp = source[start + 6:]

        end = _temp.find("\"")
        href = _temp[:end]
        start = _temp.find(">")
        _temp = _temp[start + 1:]
        end = _temp.find("<")
        name = _temp[:end]

    else:
        """
            For the entry which doesn't have the web id
            """
        #             print(source)
        start = source.find(">")
        _temp = source[start + 1:]
        end = _temp.find("<")
        name = _temp[:end]
    count = 0
    while name.find("\n") != -1 or name.find(" ") != -1 and count < 3:
        count += 1
        name = name.lstrip().rstrip()
        name = name.lstrip("\n").rstrip("\n")
    # source_btn = source_btn[:end]
    return [name.lstrip().rstrip(), href]


def find_category(soup, keywords):
    """
    ul literatureList
    -li
     -ul class literatureList_category
      -li publication
     -ul class speciesList_category
     -div class source_btn
    -li
     -ul class speciesList_category
     -div class source_btn

     sourses for link
     <li><a href="http://csdb.glycoscience.ru/bacterial/" target="_blank">XXXX</a></li>
     <li>XXXX</li>
     sourses's database
     <a class="source_btn">from JCGGDB AIST</a>
     """
    _list, _category = keywords
    species_source_list = soup.find_all('ul', {'class': _list})
    # print(species_source_list)
    database_list = list(species_source_list[0].find_all('ul', {'class': _category}))
    database_name_entry_list = list(species_source_list[0].find_all('div', {'class': "source"}))
    assert len(database_list) == len(database_name_entry_list)
    return_species_list = []
    for index, i in enumerate(database_name_entry_list):
        _entry = i.find('a', {'class': "source_btn"})
        _db_name = parse_db_name(str(_entry))
        #         print(_db_name)
        _entry_list = database_list[index].find_all("li")
        for k in _entry_list:
            _temp_entrey = parse_entry_name(str(k))
            _temp_entrey.append(_db_name)
            return_species_list.append(_temp_entrey)
    return return_species_list


def find_Descriptors(soup):
    aa = soup.find_all('code')
    entry_list = []
    for i in aa:
        entry = str(i)
        entry_list.append(entry.rstrip("</code>").lstrip("<code>"))
    return entry_list


def query_glycan(source, driver, sleep_time):
    # driver = webdriver.Firefox()
    
    print(source)
#     print("get")
    driver.get(r'https://glytoucan.org' + source+ "#Motif")
    time.sleep(sleep_time)
    # let's parse our html
#     WebDriverWait(driver, 5).until(EC.presence_of_element_located(driver.find_elements_by_xpath('//*[@id="literature"]/togostanza-literature//main/ul')))
#     print('finish wait')
    soup = BeautifulSoup(driver.page_source, "html.parser")
#     print(soup)
    a_dic = {}
    
    a_dic['species'] = find_category(soup, ['speciesList', 'speciesList_category'])
    a_dic['literature'] = find_category(soup, ['literatureList', 'literatureList_category'])
    a_dic['external'] = find_category(soup, ['externalList', 'externalList_category'])
    _templist = find_Descriptors(soup)
    if len(_templist)==4: 
        a_dic['WURCS'], a_dic['GlycoCT'], a_dic['IUPAC_Condensed'], a_dic['IUPAC_Extended'] = _templist
    elif len(_templist) == 0:
        a_dic['WURCS'], a_dic['GlycoCT'], a_dic['IUPAC_Condensed'], a_dic['IUPAC_Extended'] = "","","",""
    elif 1<=len(_templist)<=3:
        a_dic['IUPAC_Extended'] = _templist[-1]
    return a_dic

In [2]:
import platform
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import json
from selenium.webdriver.support.wait import WebDriverWait
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [17]:
driver = webdriver.Firefox()
# driver.get(r"https://glytoucan.org/Motifs/listAll")
# let's parse our html



In [4]:
motif_list_soup = BeautifulSoup(driver.page_source, "html.parser")
motif_raw_entry_list = motif_list_soup.find_all('tr')
print(motif_list_soup)

<html lang="ja"><head>
<title>Glycan Repository</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=940" name="viewport"/>
<link href="/css/bootstrap.min.css" rel="stylesheet" type="text/css"/>
<link href="//code.jquery.com/ui/1.11.1/themes/smoothness/jquery-ui.css" rel="stylesheet" type="text/css"/>
<link href="/css/GTC.css" rel="stylesheet" type="text/css"/>
<link href="/css/glycan_list_new.css" rel="stylesheet" type="text/css"/>
<link href="/favicon.ico?v=2" rel="icon" type="image/x-icon"/>
<script async="" src="//www.google-analytics.com/analytics.js"></script><script src="/js/confirmation.js" type="text/javascript"></script>
<script src="https://code.jquery.com/jquery-1.10.2.min.js" type="text/javascript"></script>
<script src="//code.jquery.com/ui/1.11.1/jquery-ui.js" type="text/javascript"></script>
<script src="/js/glycan_list_new.js" type="text/javascript"></script>
<script src="/js/init.js" type="text/javascript"></script>
<script

In [5]:
[(index, i.find('a')) for index, i in enumerate(motif_raw_entry_list)]

[(0, None),
 (1,
  <a href="/Structures/Glycans/G00055MO" target="_blank">Lactosamine motif</a>),
 (2,
  <a href="/Structures/Glycans/G00026MO" target="_blank">N-Glycan core basic</a>),
 (3,
  <a href="/Structures/Glycans/G00030MO" target="_blank">N-Glycan complex</a>),
 (4, <a href="/Structures/Glycans/G00065MO" target="_blank">VIM</a>),
 (5, <a href="/Structures/Glycans/G00051MO" target="_blank">Lewis X</a>),
 (6, <a href="/Structures/Glycans/G00068MO" target="_blank">Blood group H</a>),
 (7,
  <a href="/Structures/Glycans/G00058MO" target="_blank">Polylactosamine</a>),
 (8,
  <a href="/Structures/Glycans/G00056MO" target="_blank">Neo Lactosamine</a>),
 (9,
  <a href="/Structures/Glycans/G00046MO" target="_blank">Galalpha1-3Gal epitope</a>),
 (10,
  <a href="/Structures/Glycans/G00032MO" target="_blank">O-Glycan core 1 fuzzy</a>),
 (11, <a href="/Structures/Glycans/G00057MO" target="_blank">LacDiNAc</a>),
 (12,
  <a href="/Structures/Glycans/G00054MO" target="_blank">Sialyl Lewis X</

In [18]:
# driver = webdriver.Firefox()
# motif_dic = {}
for i in motif_raw_entry_list[53:]:
    # i=motif_raw_entry_list[1]
    # print(i)
    href_entry = i.find_all("a", href=True)
    assert len(href_entry) == 1
    a_dic = {}
    a_dic['name'], a_dic['source'] = parse_entry_name(str(href_entry))
    end = a_dic['source'].rfind("/")
    
    p_tag_entry = i.find_all("p")
    assert len(p_tag_entry) == 2
    # print(a_dic)
    passed = 0
    sleep_time=3
    while passed==0:
        try:
            print(sleep_time)
            b_dic = query_glycan(a_dic['source'], driver, sleep_time)
            passed = 1
            print('finished')
        except IndexError:
            sleep_time+=1
#         except ValueError:
#             sleep_time+=1

    b_dic['reducing_end'] = str(p_tag_entry[0]).rstrip("</p>").lstrip("<p>")
    b_dic['frequency'] = str(p_tag_entry[1]).rstrip("</p>").lstrip("<p>")
    b_dic['name'] = a_dic['name']
    b_dic['source'] = a_dic['source']
    b_dic['id']=a_dic['source'][end+1:]
    motif_dic[b_dic['id']]=b_dic


3
/Structures/Glycans/G00044MO
finished
3
/Structures/Glycans/G54161DR
finished
3
/Structures/Glycans/G00043MO
finished
3
/Structures/Glycans/G00023MO
finished
3
/Structures/Glycans/G00022MO
finished
3
/Structures/Glycans/G00060MO
finished
3
/Structures/Glycans/G00064MO
finished
3
/Structures/Glycans/G00063MO
finished
3
/Structures/Glycans/G00019MO
finished


In [20]:
assert len(motif_dic.keys())==61
with open('/Users/apple/PycharmProjects/motifs_GlyTouCan.json', 'w') as fp:
    json.dump(b_dic, fp)